In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

results_df = pd.read_csv('D:\Research Results\CSVs\\virusshare_results.csv', low_memory=False)
results_df.rename(columns={'Certificate_ID': 'certificate_ID'}, inplace=True)
certs_df = pd.read_csv('D:\Research Results\CSVs\\virusshare_certs.csv')

In [78]:
print(results_df)
print(certs_df)

                                            certificate_ID  \
0        virusshare/cs_cert_00030f454f47b70a4bf78347978...   
1        virusshare/cs_cert_00030f454f47b70a4bf78347978...   
2        virusshare/cs_cert_00030f454f47b70a4bf78347978...   
3        virusshare/cs_cert_00030f454f47b70a4bf78347978...   
4        virusshare/cs_cert_00030f454f47b70a4bf78347978...   
...                                                    ...   
1006045  virusshare/cs_cert_fffe7898116ff4541d43de96648...   
1006046  virusshare/cs_cert_fffe7898116ff4541d43de96648...   
1006047  virusshare/cs_cert_fffe7898116ff4541d43de96648...   
1006048  virusshare/cs_cert_fffe7898116ff4541d43de96648...   
1006049  virusshare/cs_cert_fffe7898116ff4541d43de96648...   

                                                 lint_name result  
0        e_subscriber_certificate_country_name_must_appear     NE  
1                            e_subject_common_name_missing     NE  
2                                e_ev_jurisdiction_

### Merge the dataframes based on certificate ID:

In [79]:
merged_df = pd.merge(results_df, certs_df, on="certificate_ID", how="left")

### Remove entries of certificates before September 21, 2016

In [80]:
merged_df['certificate_date'] = pd.to_datetime(merged_df['certificate_date'], format='%Y-%m-%d %H:%M:%S%z')
merged_df['certificate_date'] = pd.to_datetime(merged_df.certificate_date).dt.tz_localize(None)

In [81]:
merged_df.dtypes

certificate_ID                 object
lint_name                      object
result                         object
certificate_issuer             object
certificate_subject            object
certificate_date       datetime64[ns]
dtype: object

In [82]:
import datetime

mrfcsc_date = np.datetime64('2016-09-21 00:00:00')

# Drop all rows with certificates before Sep 21, 2016

filtered_df = merged_df[merged_df['certificate_date'] > mrfcsc_date]
filtered_df.sort_values(by=['certificate_date'])

,certificate_ID,lint_name,result,certificate_issuer,certificate_subject,certificate_date
396205,virusshare/cs_cert_6585261356c35371bd9852c77ee...,e_sub_cert_key_usage_cert_sign_bit_set,pass,COMODO CA Limited,AB eCommerce Inc,2016-09-22 00:00:00
610476,virusshare/cs_cert_9bd2b7f3ff4ae151a572990e21b...,e_dsa_shorter_than_2048_bits,NaN,COMODO CA Limited,Profi BINEKS,2016-09-22 00:00:00
610475,virusshare/cs_cert_9bd2b7f3ff4ae151a572990e21b...,e_sub_cert_eku_usage_legal,pass,COMODO CA Limited,Profi BINEKS,2016-09-22 00:00:00
610474,virusshare/cs_cert_9bd2b7f3ff4ae151a572990e21b...,e_sub_cert_digital_signature_not_set,pass,COMODO CA Limited,Profi BINEKS,2016-09-22 00:00:00
610473,virusshare/cs_cert_9bd2b7f3ff4ae151a572990e21b...,e_subject_organization_name_missing,pass,COMODO CA Limited,Profi BINEKS,2016-09-22 00:00:00
...,...,...,...,...,...,...
744423,virusshare/cs_cert_bdf8b59f1345b25b80cce4760a6...,e_subject_country_not_iso,pass,GlobalSign nv-sa,"SMACHNA PLITKA, TOV",2019-01-10 14:03:56
744424,virusshare/cs_cert_bdf8b59f1345b25b80cce4760a6...,e_sub_cert_key_usage_cert_sign_bit_set,pass,GlobalSign nv-sa,"SMACHNA PLITKA, TOV",2019-01-10 14:03:56
744404,virusshare/cs_cert_bdf8b59f1345b25b80cce4760a6...,e_sub_cert_key_not_critical,pass,GlobalSign nv-sa,"SMACHNA PLITKA, TOV",2019-01-10 14:03:56
744415,virusshare/cs_cert_bdf8b59f1345b25b80cce4760a6...,e_ev_business_category_missing,NaN,GlobalSign nv-sa,"SMACHNA PLITKA, TOV",2019-01-10 14:03:56


### Print Result Stats

In [83]:
from collections import Counter

errors_and_warns_df = filtered_df[(filtered_df['result'] == 'warn') | (filtered_df['result'] == 'error')]
errors_and_warns_df.shape
errors_vs_warns_count = Counter(errors_and_warns_df['result'])
print(errors_vs_warns_count)
print(Counter(filtered_df['result']))

Counter({'warn': 182, 'error': 74})
Counter({'pass': 52370, nan: 21845, 'NE': 3579, 'warn': 182, 'error': 74})


In [84]:
# Print the percentage of errors in the dataset:
errors_and_warns_df['certificate_ID'].nunique()

237

In [ ]:
### Find the most common CA's